# Blind Model Evaluation

# 1. Companies: random selection

Import Pandas, set display options, define path and name to the file containing all ratio analyses. Read the CSV file into DataFrame.

In [1]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

path = '../Sample_generation/Analyses/'
file = 'analyses_ratios'
type_file = '.csv'

sample = pd.read_csv(path+file+type_file, index_col = 0)

Random selection of 5 companies from the DataFrame.

In [2]:
random_companies = sample['company'].sample(n=5)

Define a function that receives a DataFrame and a list of companies and return a DataFrame containing only the rows from the selected companies.

In [3]:
def extract_companies(df, selected_companies):
    filtered_df = df[df['company'].isin(selected_companies)]
    
    return filtered_df

# 2. Retrieve model evaluation files

Set the path and name for the files containing the evaluation information of the Zero-shot, Few-shot, and Fine-Tuned models for the Current ratio, Quick ratio and Cash ratio. Read the CSV files into a Pandas DataFrame and utilize the 'extract_companies' function to only keep the information related to the companies that were selected randomly. Merge the DataFrames of all ratios by model type.

## 2.1 Zero-Shot

### Current ratio

In [4]:
path = './Zero_Shot_Evaluation/'
file = 'zero_shot_current_ratio'
type_file = '.csv'
zs_current = pd.read_csv(path + file + type_file, index_col=0)
zs_current = extract_companies(zs_current, random_companies)

### Quick ratio

In [5]:
path = './Zero_Shot_Evaluation/'
file = 'zero_shot_quick_ratio'
type_file = '.csv'
zs_quick = pd.read_csv(path + file + type_file, index_col=0)
zs_quick = extract_companies(zs_quick, random_companies)

### Cash ratio

In [6]:
path = './Zero_Shot_Evaluation/'
file = 'zero_shot_cash_ratio'
type_file = '.csv'
zs_cash = pd.read_csv(path + file + type_file, index_col=0)
zs_cash = extract_companies(zs_cash, random_companies)

### Concatenate files

In [7]:
zero_shot_df = pd.concat([zs_current, zs_quick, zs_cash], ignore_index=True)

## 2.2 Few-Shot

### Current ratio

In [8]:
path = './Few_Shot_Evaluation/'
file = 'few_shot_current_ratio'
type_file = '.csv'
fs_current = pd.read_csv(path + file + type_file, index_col=0)
fs_current = extract_companies(fs_current, random_companies)

### Quick ratio

In [9]:
path = './Few_Shot_Evaluation/'
file = 'few_shot_quick_ratio'
type_file = '.csv'
fs_quick = pd.read_csv(path + file + type_file, index_col=0)
fs_quick = extract_companies(fs_quick, random_companies)

### Cash ratio

In [10]:
path = './Few_Shot_Evaluation/'
file = 'few_shot_cash_ratio'
type_file = '.csv'
fs_cash = pd.read_csv(path + file + type_file, index_col=0)
fs_cash = extract_companies(fs_cash, random_companies)

### Concatenate files

In [11]:
few_shot_df = pd.concat([fs_current, fs_quick, fs_cash], ignore_index=True)

## 2.3 Fine-Tuned

### Current ratio

In [12]:
path = './Fine_Tuned_Evaluation/'
file = 'fine_tuned_current_k1_k5_similarity'
type_file = '.csv'
ft_current = pd.read_csv(path + file + type_file, index_col=0)
ft_current = ft_current.drop(['iteration', 'cosine_similarity'], axis=1)
ft_current = extract_companies(ft_current, random_companies)

### Quick ratio

In [13]:
path = './Fine_Tuned_Evaluation/'
file = 'fine_tuned_quick_k1_k5_similarity'
type_file = '.csv'
ft_quick = pd.read_csv(path + file + type_file, index_col=0)
ft_quick = ft_quick.drop(['iteration', 'cosine_similarity'], axis=1)
ft_quick = extract_companies(ft_quick, random_companies)

### Cash ratio

In [14]:
path = './Fine_Tuned_Evaluation/'
file = 'fine_tuned_cash_k1_k5_similarity'
type_file = '.csv'
ft_cash = pd.read_csv(path + file + type_file, index_col=0)
ft_cash = ft_cash.drop(['iteration', 'cosine_similarity'], axis=1)
ft_cash = extract_companies(ft_cash, random_companies)

### Concatenate files

In [15]:
fine_tuned_df = pd.concat([ft_current, ft_quick, ft_cash], ignore_index=True)

# 3. Assign random identifier

Import the uuid library to generate random identifiers. Create a DataFrame called 'model_ids'. This DataFrame will contain response identifiers and the type of model that generated the response. Define a function that will receive as parameters the DataFrame 'model_ids', the DataFrame containing the responses generated by a type of model and the name of the model type. The function removes the 'analysis_expected' column and adds the 'id_response' column at the beginning of the DataFrame containing the responses generated by the model. The function iterates over the rows of the DataFrame. In each iteration, a unique response identifier is generated. This identifier will be incorporated into the rows of the 'id_response' column, both in the DataFrame containing the model's responses and in the 'model_ids' DataFrame. Additionally, in the 'model_ids' DataFrame, each 'id_response' is assigned the name of the model type that generated it in the 'model_type' column.

In [16]:
import uuid

model_ids = pd.DataFrame(columns=['id_response', 'model_type'])

def process_dataframes(model_ids, analyses, model):
    analyses.drop(columns=['analysis_expected'], axis=1, inplace=True)
    analyses.insert(0, 'id_response', '')
    
    for index, row in analyses.iterrows():
        unique_id = uuid.uuid4()

        analyses.at[index, 'id_response'] = unique_id
        
        new_row = {'id_response': unique_id, 'model_type': model}
        
        model_ids.loc[len(model_ids)] = new_row

Call the function 'process_dataframe' using the DataFrames obtained at the end of step two, for the three types of models: Zero-shot, Few-shot and Fine-Tuned.

## 3.1 Zero-Shot

In [17]:
model = 'zero_shot'
process_dataframes(model_ids, zero_shot_df, model)

## 3.2 Few-Shot

In [18]:
model = 'few_shot'
process_dataframes(model_ids, few_shot_df, model)

## 3.3 Fine-Tuned

In [19]:
model = 'fine_tuned'
process_dataframes(model_ids, fine_tuned_df, model)

## 3.4 Random mixing of the analyses with identifiers

Randomly shuffle the DataFrames.

In [20]:
import numpy as np

dataframes = [zero_shot_df, few_shot_df, fine_tuned_df]

np.random.shuffle(dataframes)

analyses = pd.concat(dataframes, ignore_index=True)

## 3.5 Store model identifiers and model analyses with identifiers

Save the DataFrame containing the model types and the random response identifiers to a CSV file. Also, save the DataFrame containing all the analyses generated by the models with their random response identifier to a CSV file.

### Store model identifiers

In [21]:
path = './Blind_Model_Evaluation/'
file = 'model_ids'
type_file = '.csv'

model_ids.to_csv(path+file+type_file, index=True)

### Store analyses with identifiers

In [22]:
path = './Blind_Model_Evaluation/'
file = 'model_analyses_ids'
type_file = '.csv'

analyses.to_csv(path+file+type_file, index=True)

# 4. Blind Evaluation

Retrieve the file containing all the analyses generated by the models with their random response identifier.

## 4.1 Retrieve file analyses

In [23]:
path = './Blind_Model_Evaluation/'
file = 'model_analyses_ids'
type_file = '.csv'

analyses = pd.read_csv(path+file+type_file, index_col=0)

## 4.2 Define function for scoring

Define a function that called 'score' that receives the DataFrame and display each row for being scored. The function ask to the user to insert and input with the score and store it in a column called 'score'.

In [24]:
selected_columns = ['instruction', 'analysis_generated']

def score(df, columns):
    columns = columns
    df['score'] = -1
    
    for idx, row in df.iterrows():
        selected_rows_df = pd.DataFrame(row[columns]).transpose()
        display(selected_rows_df)
        user_input = input(f"Give a score (1 is correct, 0 incorrect): ")
        
        df.at[idx, 'score'] = int(user_input)
    return df

## 4.3 Score the analyses

Call the 'score' function and proceed to enter the scores for each analysis.

In [25]:
scores = score(analyses, selected_columns)

,instruction,analysis_generated
0,"Current ratio of a company is 1.49 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 1.43, placing its value in the category defined as between 1.00 and 1.49. What would be the Current ratio analysis for the years 2021 and 2022?","The company's Current ratio decreased from 1.49 in 2021 to 1.43 in 2022, indicating a reduction in its liquidity. Both ratios fall within the range of 1.00 to 1.49, suggesting that the company is able to cover its short-term debts but may not be well-prepared to handle unforeseen events affecting its liquidity."


Give a score (1 is correct, 0 incorrect):  1


,instruction,analysis_generated
1,"Current ratio of a company is 1.76 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has increased to 2.73, placing its value in the category defined as greater than 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021, the company had a comfortable buffer for unforeseen events affecting liquidity, but in 2022, the company's liquidity appears to be enhanced."


Give a score (1 is correct, 0 incorrect):  0


,instruction,analysis_generated
2,"Current ratio of a company is 1.25 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 1.11, placing its value in the category defined as between 1.00 and 1.49. What would be the Current ratio analysis for the years 2021 and 2022?","The company's Current ratio decreased from 1.25 in 2021 to 1.11 in 2022, indicating a reduction in its liquidity. Both ratios fall within the range of 1.00 to 1.49, suggesting that the company's current assets, while sufficient to cover its short-term liabilities, may not be enough to handle unforeseen events affecting its liquidity."


...

Give a score (1 is correct, 0 incorrect):  0


,instruction,analysis_generated
43,"Cash ratio of a company is 0.52 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.48, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","Cash ratio analysis for the years 2021 and 2022 would indicate that the company's liquidity position has improved in 2022 as compared to 2021.\n\nIn 2021, the cash ratio was 0.52, which is greater than the threshold of 0.40, indicating that the company had a relatively strong liquidity position. However, in 2022, the cash ratio decreased to 0.48, which is still greater than the threshold of 0.40, indicating that the company's liquidity position remained relatively strong.\n\nTherefore, the cash ratio analysis for the years 2021 and 2022 would suggest that the company's liquidity position has improved in 2022 as compared to 2021."


Give a score (1 is correct, 0 incorrect):  0


,instruction,analysis_generated
44,"Cash ratio of a company is 0.75 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.65, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","The cash ratio is a measure of a company's liquidity, calculated as current assets divided by current liabilities. A cash ratio greater than 1 indicates that a company has more current assets than current liabilities, which is a desirable position for a company. In 2021, the cash ratio was 0.75, indicating that the company had 0.75 times its current liabilities in current assets. In 2022, the cash ratio decreased to 0.65, indicating that the company had 0.65 times its current liabilities in current assets. This suggests that the company's liquidity position has weakened in 2022, as it has less current assets to cover its current liabilities."


Give a score (1 is correct, 0 incorrect):  0


## 4.4 Store analyses scores

Save the DataFrame containing the scores to a CSV file.

In [26]:
path = './Blind_Model_Evaluation/'
file = 'model_analyses_scores'
type_file = '.csv'

scores.to_csv(path+file+type_file, index=True)

# 5. Blind evaluation scores

## 5.1 Retrieve model identifiers and analyses scores files

Retrieve the file containing the random response identifiers and their correspondence with the type of model that generated the response.

In [99]:
path = './Blind_Model_Evaluation/'
file = 'model_ids'
type_file = '.csv'

model_ids = pd.read_csv(path+file+type_file, index_col=0)

Retrieve the file containing the scores.

In [100]:
path = './Blind_Model_Evaluation/'
file = 'model_analyses_scores'
type_file = '.csv'

analyses_scores = pd.read_csv(path+file+type_file, index_col=0)

## 5.2 Create functions to be applied

Create a function called 'mean_score' that calculates the average scores per model type and per ratio type for a given DataFrame. Create a function 'store_evaluation' that stores the average score in a summary template of results by model type and by ratio type.

In [101]:
def mean_score(df, ratio_type, model_type):
    filtered_df = df[(df['ratio'] == ratio_type) & (df['model_type'] == model_type)]
    
    mean = filtered_df['score'].sum() / len(filtered_df)
    formatted_mean = '{:.6f}'.format(mean)
    return formatted_mean
    
def store_evaluation(index, column_type, ratio_type, df_template, df_results):
    df_template.at[index, column_type] = mean_score(df_results, ratio_type, index)

## 5.3 Create template for storing results

Create a DataFrame template for storing the scores per model type and ratio type.

In [102]:
index = ['zero_shot', 'few_shot', 'fine_tuned']
columns = ['current_ratio', 'quick_ratio', 'cash_ratio']

scores_summary = pd.DataFrame(index=index, columns=columns)

scores_summary

,current_ratio,quick_ratio,cash_ratio
zero_shot,NaN,NaN,NaN
few_shot,NaN,NaN,NaN
fine_tuned,NaN,NaN,NaN


## 5.4 Calculate mean scores and store results

Perform a join between the information from both files using the 'id_response' field.

In [103]:
analyses_scores = pd.merge(analyses_scores, model_ids[['id_response', 'model_type']], on='id_response', how='left')

Call the 'store_evaluation' function for each model type and ratio type to calculate the average scores of the responses and store them in the results template.

### Zero-Shot

In [104]:
store_evaluation('zero_shot', 'current_ratio', 'Current', scores_summary, analyses_scores)
store_evaluation('zero_shot', 'quick_ratio', 'Quick', scores_summary, analyses_scores)
store_evaluation('zero_shot', 'cash_ratio', 'Cash', scores_summary, analyses_scores)

### Few-Shot

In [105]:
store_evaluation('few_shot', 'current_ratio', 'Current', scores_summary, analyses_scores)
store_evaluation('few_shot', 'quick_ratio', 'Quick', scores_summary, analyses_scores)
store_evaluation('few_shot', 'cash_ratio', 'Cash', scores_summary, analyses_scores)

### Fine-Tuned

In [106]:
store_evaluation('fine_tuned', 'current_ratio', 'Current', scores_summary, analyses_scores)
store_evaluation('fine_tuned', 'quick_ratio', 'Quick', scores_summary, analyses_scores)
store_evaluation('fine_tuned', 'cash_ratio', 'Cash', scores_summary, analyses_scores)

# 6. Evaluation summary

Display the summary of results by model type and ratio type."

In [107]:
scores_summary

,current_ratio,quick_ratio,cash_ratio
zero_shot,0.200000,0.600000,0.000000
few_shot,0.600000,1.000000,0.000000
fine_tuned,0.800000,0.800000,1.000000


In [108]:
path = './Evaluation_Summary/'
file = 'evaluation_summary_blind'
type_file = '.csv'

scores_summary.to_csv(path+file+type_file, index=True)